# get the data

In [1]:
import pandas as pd
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

In [4]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


# Q1

In [5]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
answer_llm = df.iloc[0].answer_llm

In [9]:
embedding_model.encode(answer_llm)[0]

-0.42244655

# Q2

In [10]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [24]:
from tqdm import tqdm
import numpy as np

evaluations = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']
    
    embedding_llm = embedding_model.encode(answer_llm)
    embedding_orig = embedding_model.encode(answer_orig)
    
    score = np.dot(embedding_llm, embedding_orig)
    evaluations.append(score)

percentile_75 = np.percentile(evaluations, 75)

# Output the result
print(percentile_75)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:51<00:00,  2.70it/s]

31.67430591583252


# Q3

In [25]:
# Function to normalize a vector
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

# Function to compute dot product (cosine similarity) between two normalized embeddings
def compute_cosine_similarity(embedding1, embedding2):
    embedding1_norm = normalize_vector(embedding1)
    embedding2_norm = normalize_vector(embedding2)
    return np.dot(embedding1_norm, embedding2_norm)


evaluations = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']
    
    embedding_llm = embedding_model.encode(answer_llm)
    embedding_orig = embedding_model.encode(answer_orig)
    
    score = compute_cosine_similarity(embedding_llm, embedding_orig)
    evaluations.append(score)

percentile_75 = np.percentile(evaluations, 75)

# Output the result
print(percentile_75)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:48<00:00,  2.75it/s]

0.8362348824739456


# Q4

In [26]:
index_10 = df[df['document'] == '5170565b'].index[0]
r = df.iloc[index_10]

In [27]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [28]:
rouge_1_f_score = scores['rouge-1']['f']
print(rouge_1_f_score)

0.45454544954545456


# Q5

In [29]:
rouge_1_f_score = scores['rouge-1']['f']
rouge_2_f_score = scores['rouge-2']['f']
rouge_l_f_score = scores['rouge-l']['f']

# Compute the average of the F1 scores
average_f_score = (rouge_1_f_score + rouge_2_f_score + rouge_l_f_score) / 3
print(average_f_score)

0.35490034990035496


# Q6

In [30]:
rouge_1_f_scores = []
rouge_2_f_scores = []
rouge_l_f_scores = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']
    
    scores = rouge_scorer.get_scores(answer_llm, answer_orig)[0]
    
    # Extract the F1 scores for rouge-1, rouge-2, and rouge-l
    rouge_1_f_scores.append(scores['rouge-1']['f'])
    rouge_2_f_scores.append(scores['rouge-2']['f'])
    rouge_l_f_scores.append(scores['rouge-l']['f'])

# Compute the average of the F1 scores
average_rouge_1_f_score = sum(rouge_1_f_scores) / len(rouge_1_f_scores)
average_rouge_2_f_score = sum(rouge_2_f_scores) / len(rouge_2_f_scores)
average_rouge_l_f_score = sum(rouge_l_f_scores) / len(rouge_l_f_scores)

# Compute the overall average of the F1 scores for rouge-1, rouge-2, and rouge-l
overall_average_f_score = (average_rouge_1_f_score + average_rouge_2_f_score + average_rouge_l_f_score) / 3
print(overall_average_f_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 320.62it/s]

0.31320536733983806
